In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
import os


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def prepare_data(data):


    # Данные для очистки.

    for i in data:

        percentiles = np.percentile(data[i],[1,99])


        data.loc[((data[i] <= percentiles[0]) |
                      (data[i] >= percentiles[1])), i] = 0


        percentiles_2 =np.percentile(data[i],[5,95])

        data.loc[((data[i] > percentiles_2[0]) &
                      (data[i] < percentiles_2[1])), i] = 0
        data.loc[(data[i] <= percentiles_2[0]), i] = -1
        data.loc[(data[i] >= percentiles_2[1]), i] = 1

        data[i] = data[i].astype('int8')




    return data

In [ ]:


for i in os.listdir("/content/drive/MyDrive/Хакатон/примеры/"):


    data = pd.read_csv(f'/content/drive/MyDrive/Хакатон/примеры/{i}/ECG.txt',
                   sep=" ", header=None).drop([7], axis=1)

    data.columns = ['I', 'II', 'V5', 'III', 'aVL', 'aVR', 'aVF']


    prepare_data(data).to_csv(f'{i}_ECG.csv')

In [ ]:
X1=[]
Y2=[]
for i in os.listdir("/content/drive/MyDrive/Хакатон/примеры/"):
  exc=pd.read_excel(f'/content/drive/MyDrive/Хакатон/примеры/{i}/dataset.xlsx')
  err=exc['тип'].tolist()
  err.pop(0)
  y=exc['арритмия'].tolist()
  y.pop(0)
  st=exc['Unnamed: 2'].tolist()
  st.pop(0)
  Y=[]
  X=[]
  z=0
  for i in y:
    if err[z]!='Z' and err[z]!='X':
      X.append(st[z])
      if i=='бигеминия':
        Y.append([1,0])
      else:
        Y.append([0,1])
    z+=1
    X1.append(X)
    Y2.append(Y)

In [ ]:

data = pd.read_csv('/content/9774_ECG.csv', index_col=0)

In [ ]:
a1=data.values
a1=list(a1)

In [ ]:
XX=[]
YY=[]

In [ ]:
XX=list(XX)
YY=list(YY)

In [ ]:
exc=pd.read_excel(f'/content/drive/MyDrive/Хакатон/примеры/9774/dataset.xlsx')
err=exc['тип'].tolist()
err.pop(0)
y=exc['арритмия'].tolist()
y.pop(0)
st=exc['Unnamed: 2'].tolist()
st.pop(0)
Y=[]
X=[]
z=0
for i in y:
  if err[z]!='Z' and err[z]!='X':
    X.append(st[z])
    if i=='бигеминия':
      Y.append([1,0])
    else:
      Y.append([0,1])
  z+=1

for i in X:
  XX.append(a1[i-1])
for i in Y:
  YY.append(i)

In [ ]:
len(YY)

1241573

In [ ]:
XX=np.array(XX)
YY=np.array(YY)
np.save('XX2',XX)
np.save('YY2',YY)

In [ ]:
XX=np.load('/content/XX2.npy')
YY=np.load('/content/YY2.npy')

In [ ]:
#перемешиваем
from sklearn import utils
x1,y1 = utils.shuffle(XX, YY)

In [ ]:
x1=list(x1)
y1=list(y1)

In [ ]:
ccc=len(y1)
ccc

1241573

In [ ]:
#уменьшаем тренировочную выборку т к значений с отсутвующей бигеминией в 10 раз юольше
x2=[]
y2=[]
w=0
for i in range(ccc):
  if w<=200000:
    x2.append(x1[i])
    y2.append(y1[i])
  else:
    if list(y1[i])==[1,0]:
      x2.append(x1[i])
      y2.append(y1[i])
  if list(y1[i])==[0,1]:
    w+=1

In [ ]:
len(x2)

375050

In [ ]:
x2=np.array(x2)
y2=np.array(y2)

In [ ]:
from tensorflow.keras.layers import Dense,SimpleRNN,Input,Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, LSTM, Input, Dropout, Embedding
import pandas as pd
import numpy as np
from sklearn import utils
import keras

In [ ]:
#структура модели
model = Sequential()
model.add(Input(shape=(7,)))
model.add(Dense(28))
model.add(Dense(64))
model.add(Dropout(0.1))
model.add(Dense(64))
model.add(Dense(28))
model.add(Dense(2, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(0.0001))

In [ ]:
history = model.fit(x2, y2, batch_size=512, epochs=15)

In [ ]:
data = pd.read_csv('/content/8555_ECG.csv', index_col=0)

In [ ]:
qw=list(data.values)

In [ ]:
n=len(qw)//250

In [ ]:
ind=[]
for i in range(n):
  if i==0:
    continue
  for j in range(20):
    ind.append(250*i+10-j)
ind.sort()

In [ ]:
sa=[]
for i in ind:
  sa.append(qw[i])

In [ ]:
x=np.array(sa)

In [ ]:
ans=model.predict(x)

In [ ]:
ans1=list(ans)

In [ ]:
A=[]
for i in ans1:
  if list(i)[0]>=0.6:
    A.append(1)
  else:
    A.append(0)

In [ ]:
indA=[]
for i in range(len(A)):
  if A[i]==1:
    indA.append(ind[i])

In [ ]:
df=pd.DataFrame({'строки':indA,})

In [ ]:
df.to_excel('/content/drive/MyDrive/Хакатон/8555.xlsx')